## FM Acquisition Example

In [ ]:
%load_ext autoreload
%autoreload 2

import numpy as np
import matplotlib.pyplot as plt
from pprint import pprint
from fibsem.fm.structures import ChannelSettings, ZParameters, FluorescenceImage
from fibsem.fm.acquisition import acquire_channels, acquire_z_stack, acquire_image
from fibsem.fm.calibration import run_autofocus, run_coarse_fine_autofocus, run_multi_position_autofocus

from fibsem.fm.microscope import FluorescenceMicroscope
# other microscopes can be imported as needed
from fibsem.fm.thermo_fisher import ThermoFisherFluorescenceMicroscope
# from fibsem.fm.odemis import OdemisFluorescenceMicroscope # -> NOTE: only works on odemis pc

# create microscope 
# fm = FluorescenceMicroscope()
# fm = OdemisFluorescenceMicroscope(None)
fm = ThermoFisherFluorescenceMicroscope()


In [ ]:
# parameters
print("Camera")
print(f"Exposure time: {fm.camera.exposure_time} s")
print(f"Pixel size: {fm.camera.pixel_size} um")
print(f"Resolution: {fm.camera.resolution} px")
print(f"Field of view: {fm.camera.field_of_view} um")
# print(f"Binning: {fm.binning}")

print("Filter Wheel")
print(f"Excitation wavelength: {fm.filter_set.excitation_wavelength} nm")
print(f"Available excitation wavelengths: {fm.filter_set.available_excitation_wavelengths}")
print(f"Emission wavelength: {fm.filter_set.emission_wavelength} nm")
print(f"Available emission wavelengths: {fm.filter_set.available_emission_wavelengths}")

print("Light Source")
print(f"Power: {fm.light_source.power} W")

In [ ]:
# Channel settings 
channel_settings = ChannelSettings(
    name="Channel-01", 
    excitation_wavelength=405, 
    emission_wavelength=None,  # None -> reflection 
    exposure_time=0.01, 
    power=0.01)

# set the channel settings
fm.set_channel(channel_settings)

# also can be set indiviudally
fm.camera.exposure_time = 0.01

In [ ]:
# QUERY: what is the best way to set emission wavelength?
# TFS: has mutlti-filter so you only set FLUORESCENCE/REFLECTION
# Odemis: has single filter so you set excitation and emission wavelength

# in practice, you'd always set the emission based on the excitation wavelength, so 
# is  it worth having a separate emission wavelength setting? aside from FL/REFLECTION


#### Objective Controls

In [ ]:
# insert objective
fm.objective.insert()

In [ ]:
# objective (lens) control
print(f"Objective Position: {fm.objective.position}")

In [ ]:
# move relative
print(f"Objective Position: {fm.objective.position}")
fm.objective.move_relative(10e-6)  # move 10 microns
print(f"Objective Position: {fm.objective.position}")

In [ ]:
# move absolute
fm.objective.move_absolute(0.006009827825)
print(f"Objective Position: {fm.objective.position}")

In [ ]:
# retract objective
fm.objective.retract()

#### Image Acquisition

In [ ]:
# single channel acquisition, uses the current microscope settings
image = fm.acquire_image()

# 2D Image (YX)
print(f"Image Shape: {image.data.shape}")

plt.imshow(image.data, cmap='gray')
plt.title(image.metadata.channels[0].name)
plt.show()


In [ ]:
# 124e-6/(4*1128)
# 4*1128

In [ ]:
image.metadata.pixel_size_x

In [ ]:
# metadata
print("Image Metadata:")
pprint(image.metadata.to_dict())


# metadata: FluorescenceImageMetadata
#     channels: List[FluorescenceChannelMetadata]

In [ ]:
# save image (ome-tiff format)
filename = "test-image.ome.tiff"
image.save(filename)

# load image
loaded_image = FluorescenceImage.load(filename) # always loaded as CZYX

# plot loaded image
plt.imshow(loaded_image.data[0, 0], cmap='gray')
plt.title(loaded_image.metadata.channels[0].name)
plt.show()

In [ ]:
print(fm.light_source.power)


fm.connection.imaging.set_active_view(3)
print(fm.connection.detector.brightness.value)
print(fm.connection.detector.camera_settings.emission.type.value)
print(fm.connection.detector.camera_settings.filter.type.value)
# print(fm.connection.detector.brightness.value)
# fm.connection.detector.brightness.value = 0.05
# print(fm.connection.detector.brightness.value)


fm.connection.detector.brightness.value = 0.005
image = fm.acquire_image()
print(fm.connection.detector.brightness.value)
plt.imshow(image.data, cmap="gray")
plt.show()

In [ ]:
fm.connection.detector.camera_settings.exposure_time.value

In [ ]:
# multi-channel acquisition
channel_settings = [
    ChannelSettings(
        name="Channel-01",
        excitation_wavelength=550,
        emission_wavelength=None,
        exposure_time=0.005,
        power=0.003),
    ChannelSettings(
        name="Channel-02",
        excitation_wavelength=550,
        emission_wavelength=550,
        exposure_time=0.50,
        power=0.3)
]

# fm.set_power(0.03)
image = acquire_channels(fm, channel_settings)

# 4D Image (CZYX)
print(f"Image Shape: {image.data.shape}")

fig, axes = plt.subplots(1, 2, figsize=(10, 5))
axes[0].imshow(image.data[0, 0, :, :], cmap='gray')
axes[0].set_title(image.metadata.channels[0].name)
axes[1].imshow(image.data[1, 0, :, :], cmap='gray')
axes[1].set_title(image.metadata.channels[1].name)

plt.show()

In [ ]:
# generate z-parameters (objective positions)
zparams = ZParameters(zmin=-2e-6, zmax=2e-6, zstep=1e-6)

zpositions = zparams.generate_positions(z_init=fm.objective.position)
print(f"Z-Positions: {zpositions}")

In [ ]:
# acquire a z-stack
# NOTE: make sure the objective is inserted and in a safe position
image  = acquire_image(fm, channel_settings, zparams)

# 4D Image (CZXY)
print(f"Image Shape: {image.data.shape}")

# plot the maximum intensity projection for each channel
fig, axes = plt.subplots(1, 2, figsize=(10, 5))
axes[0].imshow(np.max(image.data[0, :, :, :], axis=0), cmap='gray')
axes[0].set_title(image.metadata.channels[0].name)
axes[1].imshow(np.max(image.data[1, :, :, :], axis=0), cmap='gray')
axes[1].set_title(image.metadata.channels[1].name)

plt.show()

In [ ]:
mip = image.max_intensity_projection(channel=0, return_2d=True)
plt.imshow(mip, cmap='gray')
plt.title("Maximum Intensity Projection")
plt.show()

In [ ]:
focus_stack = image.focus_stack()

print(f"Focus Stack Shape: {focus_stack.data.shape}")

# plot the focus stack
fig, axes = plt.subplots(1, 2, figsize=(10, 5))
axes[0].imshow(focus_stack.data[0, 0, :, :], cmap='gray')
axes[0].set_title(focus_stack.metadata.channels[0].name)
axes[1].imshow(focus_stack.data[1, 0, :, :], cmap='gray')
axes[1].set_title(focus_stack.metadata.channels[1].name)
plt.show()


#### Live Acquisition

In [ ]:
import time

def on_acquisition_signal(image: FluorescenceImage):
    print("Acquisition signal received!")
    print(f"Image shape: {image.data.shape}")
    print(f"Acquisition Date: {image.metadata.acquisition_date}")
    print(f"Channel Name: {image.metadata.channels[0].name}")
    print(f"Excitation Power: {image.metadata.channels[0].power} W")
    print(f"Exposure Time: {image.metadata.channels[0].exposure_time} s")
    print(f"Excitation Wavelength: {image.metadata.channels[0].excitation_wavelength} nm")
    print(f"Emission Wavelength: {image.metadata.channels[0].emission_wavelength} nm")
    print(f"Objective Magnification: {image.metadata.channels[0].objective_magnification}x")
    print(f"Objective Position: {image.metadata.channels[0].objective_position:.2e} m")
    print(f"Image Data Type: {image.data.dtype}")
    print("-"*80)
    # plt.imshow(image.data, cmap='gray')
    # plt.show()

# acquisition emits a FluorescenceImage signal once image is acquire, can be subscribed to
fm.acquisition_signal.disconnect() # disconnect any previous connections
fm.acquisition_signal.connect(on_acquisition_signal)

fm.set_exposure_time(0.005)

In [ ]:
# start the acquisition
fm.start_acquisition()

time.sleep(5)

# stop acquisition
fm.stop_acquisition()

#### AutoFocus

In [ ]:
# NOTE: make sure the objective is inserted, and safe before running autofocus
channel_settings = channel_settings[0]
image1 = acquire_image(fm, channel_settings)
best_focus_position = run_autofocus(fm)
print(f"Best Focus Position: {best_focus_position}")

image2 = acquire_image(fm, channel_settings)

# plot the acquired images
fig, axes = plt.subplots(1, 2, figsize=(10, 5))
axes[0].imshow(image1.data[0, 0, :, :], cmap='gray')
axes[0].set_title(image1.metadata.channels[0].name)
axes[1].imshow(image2.data[0, 0, :, :], cmap='gray')
axes[1].set_title(image2.metadata.channels[0].name)

plt.show()

In [ ]:
# NOTE: make sure the objective is inserted, and safe before running autofocus
image1 = acquire_channels(fm, channel_settings)

best_focus_position = run_coarse_fine_autofocus(fm)
print(f"Best Focus Position: {best_focus_position}")

image2 = acquire_channels(fm, channel_settings)

# plot the acquired images
fig, axes = plt.subplots(1, 2, figsize=(10, 5))
axes[0].imshow(image1.data[0, 0, :, :], cmap='gray')
axes[0].set_title(image1.metadata.channels[0].name)
axes[1].imshow(image2.data[0, 0, :, :], cmap='gray')
axes[1].set_title(image2.metadata.channels[0].name)

plt.show()

In [ ]:
# requires integrated fibsem/fm microscope
from fibsem import utils
from fibsem.structures import BeamType
microscope, settings = utils.setup_session()


# create some test positions (project-stable-move)
current_position = microscope.get_stage_position()
stages_positions = [
    microscope.project_stable_move(dx=-50e-6, dy=-50e-6, beam_type=BeamType.ELECTRON, base_position=current_position),
    microscope.project_stable_move(dx=50e-6, dy=-50e-6, beam_type=BeamType.ELECTRON, base_position=current_position),
    microscope.project_stable_move(dx=0, dy=0, beam_type=BeamType.ELECTRON, base_position=current_position),
    microscope.project_stable_move(dx=-50e-6, dy=50e-6, beam_type=BeamType.ELECTRON, base_position=current_position),
    microscope.project_stable_move(dx=50e-6, dy=50e-6, beam_type=BeamType.ELECTRON, base_position=current_position),
]

# set names for the positions
stages_positions[0].name = "top-left"
stages_positions[1].name = "top-right"
stages_positions[2].name = "center"
stages_positions[3].name = "bottom-left"
stages_positions[4].name = "bottom-right"

# run multi-position autofocus
# focus_map = run_multi_position_autofocus(microscope, stages_positions, return_to_start=True)



#### Overview Acquisition

In [ ]:
fm.acquisition_signal.disconnect()

In [ ]:
%load_ext autoreload
%autoreload 2


from fibsem import utils
from fibsem.fm.acquisition import acquire_tileset, ChannelSettings, acquire_and_stitch_tileset, ZParameters, stitch_tileset, AutofocusMode
from fibsem.structures import BeamType, FibsemStagePosition
import matplotlib.pyplot as plt
from pprint import pprint

CONFIG_PATH = r"C:\Users\User\Documents\github\openfibsem\fibsem-os\fibsem\config\tfs-arctis-configuration.yaml"
microscope, settings = utils.setup_session(config_path=CONFIG_PATH)

In [ ]:
image1 = microscope.fm.acquire_image()
# microscope.stable_move(dx=50e-6, dy=0e-6, beam_type=BeamType.ELECTRON)
image2 = microscope.fm.acquire_image()

# plot the acquired images
fig, axes = plt.subplots(1, 2, figsize=(10, 5))
axes[0].imshow(image1.data, cmap='gray')
axes[0].set_title(f"Before: {image1.metadata.channels[0].name}")
axes[1].imshow(image2.data, cmap='gray')
axes[1].set_title(f"After: {image2.metadata.channels[0].name}")

plt.show()

In [ ]:
channel_settings = ChannelSettings(
    name="Channel-01",
    excitation_wavelength=550,
    emission_wavelength=None,
    power=0.03,
    exposure_time=0.005,
)

# To acquire a tileset on arctis:
# pre-tilt = 0
# orientation=FM
# beam_type=BeamType.ELECTRON

microscope.system.stage.shuttle_pre_tilt = 0
microscope.stage_is_compustage = True
# microscope.move_flat_to_beam(beam_type=BeamType.ELECTRON)
print(f"Stage Position: {microscope.get_stage_position()}")
print(f"Stage Orientation: {microscope.get_stage_orientation()}")
print(f"FM Objective Position: {microscope.fm.objective.position}")
beam_type = BeamType.ELECTRON


In [ ]:
# acquire a tileset
stitched_image = acquire_and_stitch_tileset(
    microscope=microscope,
    channel_settings=channel_settings,
    grid_size=(5, 5),
    tile_overlap=0.1,
    beam_type=beam_type,
    # zparams=ZParameters(zmin=-5.0e-6, zmax=5.0e-6, zstep=2.5e-6),
    autofocus_mode=AutofocusMode.EACH_TILE,
    autofocus_zparams=ZParameters(zmin=-15e-6, zmax=15e-6, zstep=1.5e-6)
)
print(microscope.get_stage_position())

In [ ]:
# plot the stitched image
if stitched_image.data.ndim == 4:
    dat = stitched_image.data[0, 0]
else:
    dat=stitched_image.data
plt.imshow(dat, cmap='gray')
plt.title("Stitched Image from Tileset")
plt.colorbar()
plt.show()

import datetime
filename = f"overview-5x5-autofocus.ome.tiff"
stitched_image.save(filename)
pprint(stitched_image.metadata.to_dict())

#### Move to Microscope
Note: Need to take pre-caution when running this as it will insert the objective


In [ ]:
# requires integrated fibsem/fm microscope
from fibsem import utils
from fibsem.structures import BeamType
microscope, settings = utils.setup_session()


In [ ]:
# current orientation
orientation = microscope.get_stage_orientation()
print(f"Current Orientation: {orientation}")

# if orientation != "FIB":
    # microscope.move_flat_to_beam(BeamType.ION)

In [ ]:
# microscope.move_to_microscope("FIBSEM")

print(f"Moved to FM orientation: {microscope.get_stage_orientation()}")

In [ ]:
print(microscope.get_stage_position())

#### Focus Stacking

In [ ]:
from fibsem.fm.calibration import create_block_based_focus_stack, create_pixel_based_focus_stack
import tifffile as tff
import matplotlib.pyplot as plt
import numpy as np

PATH = "/home/patrick/github/3DCT/3D_correlation_test_dataset/test-image2.ome.tiff"

image = tff.imread(PATH)
print(f"Image Shape: {image.shape}")

for i in range(image.shape[0]):
    print(f"Channel {i}: Shape {image[i].shape}")
    channel_data = image[i]
    stacked_image = create_block_based_focus_stack(channel_data, method='tenengrad', block_size=256, smooth_transitions=True)
    pixel_stacked_image = create_pixel_based_focus_stack(channel_data, method='tenengrad')
    fig, ax = plt.subplots(ncols=3, figsize=(15, 10))
    ax[0].imshow(np.max(channel_data, axis=0), cmap='gray')
    ax[0].set_title("Maximum Intensity Projection")
    ax[1].imshow(stacked_image, cmap='gray')
    ax[1].set_title("Focus Stack Projection")
    ax[2].imshow(pixel_stacked_image, cmap='gray')
    ax[2].set_title("Pixel-Based Focus Stack Projection")

    plt.tight_layout()
    plt.show()
